In [3]:
from gensim_preprocessing import corpus, dictionary, texts
import numpy as np
import json
import glob
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import pyLDAvis
import pyLDAvis.gensim
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [4]:
bigrams_phrases = gensim.models.Phrases(texts, min_count=5, threshold=100)
trigram_phrases = gensim.models.Phrases(bigrams_phrases[texts], threshold=100)

bigram = gensim.models.phrases.Phraser(bigrams_phrases)
trigram = gensim.models.phrases.Phraser(trigram_phrases)

def make_bigrams(textss):
    return [bigram[doc] for doc in textss]

def make_trigrams(textss):
    return [trigram[bigram[doc]] for doc in textss]

data_bigrams = make_bigrams(texts)
data_bigrams_trigrams = make_trigrams(data_bigrams)

print(data_bigrams_trigrams[0])


['pelo', 'primeirer', 'vez', 'velho_moinho', 'lodando_vento', 'venho_aqui', 'recorrer_esse', 'momento_todo_visto_com', 'abraçao_essa', 'mulher_nunca_encontrei', 'moinho', 'igual', 'vida', 'igual', 'ao', 'ano', 'vento', 'desfaça', 'lugar', 'destino', 'amor', 'tristeza', 'velho_moinho', 'lodando_vento', 'venho_aqui', 'recorrer_esse', 'momento_todo_visto_com', 'abraçao_essa', 'mulher_nunca_encontrei', 'velho_moinho', 'lodando_vento', 'venho_aqui', 'recorrer_esse', 'momento_todo_visto_com', 'abraçao_essa', 'mulher_nunca_encontrei', 'patra', 'como', 'podama', 'musiador', 'vejo', 'vida_mas', 'tal', 'sorte', 'foi', 'destino', 'mulher', 'crida', 'chegou', 'tempo', 'tudo', 'hora', 'com', 'motor', 'iver', 'não', 'restam', 'cinza', 'onde', 'coraçao', 'não', 'sinto', 'falso', 'tal', 'baixo', 'minha', 'ilusão', 'nem', 'chorar', 'queria', 'chorar_mas', 'não_tenho', 'pra_duplo', 'por', 'seva_marcar', 'eler', 'não_vai', 'contrar', 'adorava', 'como', 'toder', 'moceador', 'vejo', 'vida_mas', 'tal', 'sor

In [ ]:
# TF-IDF REMOVAL

In [5]:
from gensim.models import TfidfModel

id2word = corpora.Dictionary(data_bigrams_trigrams)

texts_removal = data_bigrams_trigrams

corpus = [id2word.doc2bow(text) for text in texts_removal]
print(corpus[0][0:20])

tfidf = TfidfModel(corpus, id2word=id2word )

low_value = 0.03
words = []
words_missing_in_tfidf = []


[(0, 3), (1, 1), (2, 1), (3, 1), (4, 2), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 8), (12, 1), (13, 5), (14, 1), (15, 4), (16, 2), (17, 1), (18, 1), (19, 1)]


In [6]:
print(texts[0][0:20])

['pelo', 'primeirer', 'vez', 'velho', 'moinho', 'lodando', 'vento', 'venho', 'aqui', 'recorrer', 'esse', 'momento', 'todo', 'visto', 'com', 'abraçao', 'essa', 'mulher', 'nunca', 'encontrei']


## Vizualizing the data

In [8]:
lda_model = gensim.models.ldamodel.LdaModel(corpus = corpus,
                                           id2word=id2word,
                                           num_topics=15,
                                           random_state = 100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=15,
                                           alpha="auto")

In [9]:
pyLDAvis.enable_notebook()
vis=pyLDAvis.gensim.prepare(lda_model, corpus, id2word, mds="mmds", R=30)
vis

/Users/mariemccormick/AciduleApp/lib/python3.11/site-packages/sklearn/manifold/_mds.py:299: FutureWarning: The default value of `normalized_stress` will change to `'auto'` in version 1.4. To suppress this warning, manually set the value of `normalized_stress`.
  warnings.warn(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
9     -0.132335 -0.120116       1        1  23.265059
13    -0.216183 -0.064793       2        1  22.948137
5     -0.195616 -0.172355       3        1  13.988566
0     -0.140368 -0.266070       4        1  10.210797
1     -0.055397 -0.214099       5        1   8.302455
3     -0.049118 -0.114991       6        1   5.196137
11    -0.154963  0.008491       7        1   4.969130
2     -0.302840  0.008473       8        1   4.852054
4      0.135737 -0.206818       9        1   2.980773
8      0.082827  0.440363      10        1   1.530380
14    -0.098082  0.210928      11        1   1.048891
6      0.350792  0.013459      12        1   0.295463
12     0.299392  0.121627      13        1   0.205650
10     0.240811  0.253441      14        1   0.188369
7      0.235345  0.102461      15        1   0.018138, topic_info=           Term         Freq        Total Category  logprob  loglift
3720      femme  1208.000000  1208.000000  Default  30.0000  30.0000
1223   logement  1503.000000  1503.000000  Default  29.0000  29.0000
662    monsieur  7508.000000  7508.000000  Default  28.0000  28.0000
500        bien  6701.000000  6701.000000  Default  27.0000  27.0000
1392   quartier  1566.000000  1566.000000  Default  26.0000  26.0000
...         ...          ...          ...      ...      ...      ...
5440      hôtel     0.168848   144.486698  Topic15  -6.6483   1.8630
616       image     0.114371   253.063789  Topic15  -7.0378   0.9130
3339    qualité     0.115251   401.281130  Topic15  -7.0301   0.4596
1030     direct     0.115055   675.451624  Topic15  -7.0318  -0.0628
4471  industrie     0.096128    66.419723  Topic15  -7.2116   2.0768

[1056 rows x 6 columns], token_table=       Topic      Freq      Term
term                            
1560       2  0.462726  aacidule
1560       3  0.132207  aacidule
1560       5  0.330519  aacidule
851        1  0.163850  accepter
851        2  0.153414  accepter
...      ...       ...       ...
7599       2  0.182905       île
7599       3  0.060968       île
7599       5  0.690976       île
16144      1  0.034370      îlot
16144      5  0.927990      îlot

[2808 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[10, 14, 6, 1, 2, 4, 12, 3, 5, 9, 15, 7, 13, 11, 8])